In [1]:
# ============================================================
# PYOMO FIRST MODEL (MINI SUPPLY CHAIN)
# Purpose: learn Pyomo syntax end-to-end on the smallest case
# Decision: how much to ship from Factory -> Customers
# Objective: minimize shipping cost
# Constraints: supply limit + meet demand
# ============================================================

from pyomo.environ import (
    ConcreteModel, Set, Param, Var,
    NonNegativeReals, Objective, Constraint,
    minimize, SolverFactory, value
)

# ----------------------------
# 1) DATA (plain Python)
# ----------------------------
customers = ["C1", "C2"]

supply = 100
demand = {"C1": 30, "C2": 60}
cost   = {"C1": 2.0, "C2": 3.0}   # cost per unit shipped to each customer

# ----------------------------
# 2) MODEL SHELL
# ----------------------------
m = ConcreteModel(name="First_Pyomo_Model")

# ----------------------------
# 3) SETS
# ----------------------------
m.C = Set(initialize=customers)   # Customers set

# ----------------------------
# 4) PARAMETERS
# ----------------------------
m.supply = Param(initialize=supply)                         # single number
m.demand = Param(m.C, initialize=lambda mdl, c: demand[c])  # indexed by customer
m.cost   = Param(m.C, initialize=lambda mdl, c: cost[c])    # indexed by customer

# ----------------------------
# 5) DECISION VARIABLES
# ----------------------------
# x[c] = units shipped to customer c
m.x = Var(m.C, domain=NonNegativeReals)

# ----------------------------
# 6) OBJECTIVE FUNCTION
# ----------------------------
def total_cost(mdl):
    return sum(mdl.cost[c] * mdl.x[c] for c in mdl.C)

m.obj = Objective(rule=total_cost, sense=minimize)

# ----------------------------
# 7) CONSTRAINTS
# ----------------------------

# (A) Supply: total shipped <= supply
def supply_limit(mdl):
    return sum(mdl.x[c] for c in mdl.C) <= mdl.supply

m.supply_con = Constraint(rule=supply_limit)

# (B) Demand: ship at least demand to each customer
def meet_demand(mdl, c):
    return mdl.x[c] >= mdl.demand[c]

m.demand_con = Constraint(m.C, rule=meet_demand)

# ----------------------------
# 8) SOLVE
# ----------------------------
# You need an LP solver installed, e.g.:
# - glpk (free)
# - cbc  (free)
# - highs (fast, free)
#
# Change "glpk" to the solver you have.
solver = SolverFactory("glpk")
results = solver.solve(m, tee=False)

# ----------------------------
# 9) READ RESULTS
# ----------------------------
print("\n=== SOLUTION ===")
print("Total cost:", value(m.obj))
for c in m.C:
    print(f"Ship to {c}: {value(m.x[c])}")

print("\nSlack supply (unused):", value(m.supply) - sum(value(m.x[c]) for c in m.C))


=== SOLUTION ===
Total cost: 240.0
Ship to C1: 30.0
Ship to C2: 60.0

Slack supply (unused): 10.0
